There must be a way to effectively use a model without being redundant to the actual data present. Such as, we want the model to calculate the NDVI of the area itself, on aggregate or average

That means give it the access to make the exact computations that it needs to
you have the basic functions down.
just make more. simple, complex, dynamic. and when you are able to abstract, then you continue on

so what you are developing are FUNCTIONS. that do data query, postGIS and whatever else under the hood. then GIVING those functions to an LLM

It is best to just save all of the data of lamwo somewhere. best not to lag.
But how do i design the dataset such that the query of regions, and performing of a function over a specific region is not causing serious overhead in time + memory

So i can keep an index of all the tiles in Lamwo, and this is what is accessed first in the notebook, so that the locations are figured, buffered, whatever they need to be. then the indices are intersected and these will then be used to load the tiles and perform the necessary analysis.

we can alternatively try out the Google Earth Engine direct query. this is way less complex for me. but i don't know if we would be able to something like that ultimately due to licensing and whatnot - these things are normally used for research purposes



In [1]:
# prompt: initialize ee and mount google drive
from google.colab import drive
drive.mount('/content/drive')

!git clone https://github.com/nasaharvest/galileo.git
!cd galileo
!pip install wandb einops torch torchvision earthengine-api rioxarray rasterio tqdm geopandas fiona geobench h5py scikit-learn types-requests

Mounted at /content/drive
Cloning into 'galileo'...
remote: Enumerating objects: 405, done.
remote: Counting objects: 100% (34/34), done.
remote: Compressing objects: 100% (27/27), done.
remote: Total 405 (delta 13), reused 21 (delta 6), pack-reused 371 (from 2)
Receiving objects: 100% (405/405), 44.83 MiB | 23.33 MiB/s, done.
Resolving deltas: 100% (60/60), done.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.6/56.6 kB 2.5 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of xarray to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of xarray to determine which version is compatible with other requirements. This could take a while.
INFO: This is taking longer than usual. You might need to provide the dependency resolver with stricter constraints to reduce runtime. See https://pip.pypa.io/warnings/backtracking for guidance. If you want to abort this run, press Ctrl + C.
     ━━━━━━━━━━━━━━

In [2]:
# Reinstalling numpy and pandas with specific versions to ensure compatibility:
!pip install numpy==1.25.2
!pip install pandas==2.0.3

# for evals
!pip install fast_pytorch_kmeans satlaspretrain-model timm breizhcrops

  Using cached fast_pytorch_kmeans-0.2.2-py3-none-any.whl.metadata (1.1 kB)
ERROR: Could not find a version that satisfies the requirement satlaspretrain-model (from versions: none)
ERROR: No matching distribution found for satlaspretrain-model


In [1]:
# 0) (Only once) install Galileo in your Colab env
!pip install git+https://github.com/nasaharvest/galileo.git

  Cloning https://github.com/nasaharvest/galileo.git to /tmp/pip-req-build-fvfpm8rp
  Running command git clone --filter=blob:none --quiet https://github.com/nasaharvest/galileo.git /tmp/pip-req-build-fvfpm8rp
  Resolved https://github.com/nasaharvest/galileo.git to commit e0581f735763fa5752b1d07b378384a3f28ca697
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for UNKNOWN: filename=unknown-0.0.0-py3-none-any.whl size=208222 sha256=ecf570e41a27a104250f58358aa6ef474b2cb18d1bdbfd954fcaff058370e306
  Stored in directory: /tmp/pip-ephem-wheel-cache-3mvnv7ku/wheels/c2/2f/ad/1a7e58d65f5c119f28826d87415284fdd8b16133f4d092fe6e
Successfully built UNKNOWN


In [3]:
import ee
import os
import numpy as np
import geopandas as gpd
import io
import requests
# Trigger the authentication flow.
ee.Authenticate()

# Initialize the library.
ee.Initialize(project='ee-isekalala')

In [4]:
district_path = '/content/drive/Shareddrives/Sunbird AI/Projects/GIZ Mini-grid Identification/Phase II/Data/administrative areas/UGA Lamwo district.gpkg'
gdf = gpd.read_file(district_path)
geometry = gdf.iloc[0]['geometry']
print(geometry)

MULTIPOLYGON (((33.329295171589 3.75417023573826, 33.329662479234 3.72340178089165, 33.3125614766473 3.70575760684994, 33.3182324601876 3.68956169338361, 33.3028979425049 3.67030626941849, 33.2937432150377 3.64271632008894, 33.2939892914418 3.6240163074893, 33.3074077295729 3.57518576414918, 33.2896836734715 3.57141899509674, 33.2758682922742 3.56091074369521, 33.2678491853448 3.52985500325344, 33.2708353847752 3.51602389584785, 33.238030146681 3.51322844909065, 33.1840129781047 3.5219322148051, 33.1486630814069 3.50704718389953, 33.1332699702114 3.50987218416526, 33.1223056665544 3.49952389713805, 33.1121029599852 3.49715783998949, 33.0452854957029 3.52005567966441, 33.0296140729577 3.5101179074551, 33.0183591346619 3.48844149238095, 33.0038064234659 3.47949183570515, 32.9751583639278 3.48155286676052, 32.9650295869098 3.45665228200391, 32.9506351921046 3.44445222048598, 32.9505472874624 3.43600227574596, 32.9400765155276 3.43817296445668, 32.9168275482393 3.42707216019263, 32.9159851

In [5]:
def geom_to_ee_fmt(geometry):
  # Convert the geometry to a GeoJSON format
  geometry_geojson = geometry.__geo_interface__

# Ensure it is in the format expected by ee.Geometry.MultiPolygon
  if geometry_geojson["type"] == "MultiPolygon":
    # Create an ee.Geometry.MultiPolygon
    ee_geometry = ee.Geometry.MultiPolygon(geometry_geojson["coordinates"])
  elif geometry_geojson["type"] == "Polygon":
    # If it's a single Polygon, convert it to MultiPolygon
    ee_geometry = ee.Geometry.MultiPolygon([geometry_geojson["coordinates"]])
  else:
    raise ValueError("Geometry type is not Polygon or MultiPolygon.")
  return ee_geometry


# ee_geometry is now an Earth Engine MultiPolygon geometry
lamwo = geom_to_ee_fmt(geometry)
print(lamwo.getInfo())

{'type': 'MultiPolygon', 'coordinates': [[[[33.329295171589, 3.75417023573826], [33.329662479234, 3.72340178089165], [33.3125614766473, 3.70575760684994], [33.3182324601876, 3.68956169338361], [33.3028979425049, 3.67030626941849], [33.2937432150377, 3.64271632008894], [33.2939892914418, 3.6240163074893], [33.3074077295729, 3.57518576414918], [33.2896836734715, 3.57141899509674], [33.2758682922742, 3.56091074369521], [33.2678491853448, 3.52985500325344], [33.2708353847752, 3.51602389584785], [33.238030146681, 3.51322844909065], [33.1840129781047, 3.5219322148051], [33.1486630814069, 3.50704718389953], [33.1332699702114, 3.50987218416526], [33.1223056665544, 3.49952389713805], [33.1121029599852, 3.49715783998949], [33.0452854957029, 3.52005567966441], [33.0296140729577, 3.5101179074551], [33.0183591346619, 3.48844149238095], [33.0038064234659, 3.47949183570515], [32.9751583639278, 3.48155286676052], [32.9650295869098, 3.45665228200391], [32.9506351921046, 3.44445222048598], [32.950547287

In [9]:
# -----------------------------------------------------------------------------
# 2) PARAMETERS – adjust as needed for reproducibility
# -----------------------------------------------------------------------------

NUM_SAMPLES        = 300       # number of random points
YEAR               = 2024      # year for NDVI time series
NDVI_THRESHOLD     = 0.4       # median NDVI > threshold → high vegetation
CLOUD_CUTOFF       = 20        # max cloud percentage per image
EXPORT_FOLDER      = "earthengine_exports"
EXPORT_PREFIX      = f"veg_samples_{YEAR}"

# -----------------------------------------------------------------------------
# 3) DEFINE YOUR REGION AND RANDOM POINTS
# -----------------------------------------------------------------------------
region = lamwo
points = ee.FeatureCollection.randomPoints(
    region,
    NUM_SAMPLES,
    seed=42
)

# -----------------------------------------------------------------------------
# 4) BUILD A YEAR-LONG NDVI IMAGE COLLECTION
# -----------------------------------------------------------------------------
# Sentinel-2 SR, filter cloudy scenes, select Red (B4) & NIR (B8)
s2 = (
    ee.ImageCollection("COPERNICUS/S2_SR")
      .filterBounds(region)
      .filterDate(f"{YEAR}-01-01", f"{YEAR}-12-31")
      .filter(ee.Filter.lt("CLOUDY_PIXEL_PERCENTAGE", CLOUD_CUTOFF))
      .select(["B4", "B8"])
)

# Map over each image to add an 'NDVI' band
def add_ndvi(img):
    ndvi = img.normalizedDifference(["B8", "B4"]).rename("NDVI")
    return img.addBands(ndvi)

s2_ndvi = s2.map(add_ndvi)

# -----------------------------------------------------------------------------
# 5) REDUCE TO A SINGLE MEDIAN-NDVI IMAGE
# -----------------------------------------------------------------------------
ndvi_med = s2_ndvi.select("NDVI").median()

# -----------------------------------------------------------------------------
# 6) SAMPLE THE MEDIAN NDVI AT EACH RANDOM POINT
# -----------------------------------------------------------------------------
samples = ndvi_med.sampleRegions(
    collection=points,
    properties=[],     # no extra properties
    scale=10,          # Sentinel-2 native resolution
    geometries=True    # keep point geometries in export
)

# -----------------------------------------------------------------------------
# 7) LABEL BASED ON MEDIAN NDVI THRESHOLD
# -----------------------------------------------------------------------------
def label_feature(feat):
    nd = ee.Number(feat.get("NDVI"))
    label = nd.gt(NDVI_THRESHOLD)  # 1 if NDVI > threshold, else 0
    return feat.set({
        "median_NDVI": nd,
        "label": label
    })

labeled = samples.map(label_feature)

# -----------------------------------------------------------------------------
# 8) EXPORT THE LABELED SAMPLES USING getDownloadURL
# -----------------------------------------------------------------------------
url = labeled.getDownloadURL(**{
    'filetype': 'CSV',
    'filename': EXPORT_PREFIX  # You can adjust the filename as needed
})

print(f"Download URL: {url}")

# You can then use this URL to download the data, for example:
response = requests.get(url)
with open(f'{EXPORT_PREFIX}.csv', 'wb') as f:
    f.write(response.content)

print(f"🚀 Export completed. Data saved to {EXPORT_PREFIX}.csv")

Download URL: https://earthengine.googleapis.com/v1/projects/ee-isekalala/tables/8561ebe1eedab19f74f05498802e1dd3-9eb1a9a2b3a12424186b0077ace5284d:getFeatures
🚀 Export completed. Data saved to veg_samples_2024.csv


In [10]:
def ee_bytes_to_img(pixel_data):
    # no more num_channels check
    return np.stack([pixel_data[field] for field in pixel_data.dtype.names], axis=-1)


In [11]:
import ee
import pandas as pd
from pathlib import Path
import geopandas as gpd
from shapely.geometry import Point
import json

# -----------------------------------------------------------------------------
# 1) CONFIGURATION
# -----------------------------------------------------------------------------
# Path to the CSV you exported (must have 'longitude','latitude','label' columns)
CSV_PATH       = "veg_samples_2024.csv"
# Prefix for exported filenames
FILE_PREFIX    = "veg_patch"
#
output_dir = 'content/patches'
os.makedirs(output_dir, exist_ok=True)

# Patch size in pixels and Sentinel-2 scale (10 m)
PATCH_PIXELS   = 64
SCALE          = 10  # meters per pixel
# List of Sentinel-2 bands matching Galileo’s S2_BANDS
S2_BANDS = ["B2","B3","B4","B5","B6","B7","B8","B8A","B11","B12"]


# -----------------------------------------------------------------------------
# 3) LOAD YOUR SAMPLES INTO AEE FEATURECOLLECTION
# -----------------------------------------------------------------------------
df = pd.read_csv(CSV_PATH)
# Creating geometry column using shapely.geometry.Point
df['geometry'] = df.apply(lambda row: Point(json.loads(row['.geo'])["coordinates"][0], json.loads(row['.geo'])["coordinates"][1]), axis=1) # Creating a geometry column from longitude and latitude
crs = {'init': 'epsg:4326'}
gdf = gpd.GeoDataFrame(df, crs="EPSG:4326", geometry='geometry') # Using the new 'geometry' column

# Build a FeatureCollection of points with label as property
features = []
for idx, row in df.iterrows():
    lon, lat = row.geometry.x, row.geometry.y # Accessing coordinates from geometry column
    pt = ee.Geometry.Point([lon, lat])
    feat = ee.Feature(pt, {
        "sample_id": int(idx),
        "label": int(row.label)
    })
    features.append(feat)
samples_fc = ee.FeatureCollection(features)

# -----------------------------------------------------------------------------
# 4) PREPARE MONTHLY MEDIAN COMPOSITES
# -----------------------------------------------------------------------------

def monthly_composites(region, year):
    """Return a list of 12 ee.Image composites, one per month."""
    def make_month(m):
        start = ee.Date.fromYMD(year, m, 1)
        end   = start.advance(1, "month")
        coll = (
            ee.ImageCollection("COPERNICUS/S2_SR_HARMONIZED")
              .filterBounds(region)
              .filterDate(start, end)
              .filter(ee.Filter.lt("CLOUDY_PIXEL_PERCENTAGE", 20))
              .select(S2_BANDS)
        )
        # median composite for that month
        img = coll.median()
        # rename bands to include month index, e.g. "B4_m01"
        month_str = ee.Number(m).format("%02d")
        new_names = img.bandNames().map(lambda b: ee.String(b).cat("_m").cat(month_str))
        return img.rename(new_names)

    months = list(range(1, 13))
    return [make_month(m) for m in months]



# -----------------------------------------------------------------------------
# 5) EXPORT PATCHES LOOP
# -----------------------------------------------------------------------------

def process_patch(feat, patch_name):
    sid = feat.get("sample_id")
    geom = ee.Geometry(feat['geometry'])  # Access geometry from dictionary
    # compute bounding box around point: halfWidth = (PATCH_PIXELS/2) * SCALE
    half_m = (PATCH_PIXELS/2) * SCALE
    region = geom.buffer(half_m).bounds()
    monthly_imgs = monthly_composites(region, year=2024)
    # stack into one multiband image
    stacked = ee.Image.cat(monthly_imgs)
    #get band names of stacked
    band_names = stacked.bandNames().getInfo()

    url = stacked.getDownloadUrl({
      'bands': band_names,
      'region': region,
      'dimensions': '64x64',
      'format': 'NPY'
    })

    response = requests.get(url)
    data = np.load(io.BytesIO(response.content), allow_pickle=True)
    img_full = ee_bytes_to_img(data)      # (64,64,120)
    # reshape to (12,64,64,10) for Galileo:
    H, W, Ctot = img_full.shape
    C = len(S2_BANDS)                     # 10
    T = Ctot // C                         # 12
    galileo_input = img_full.reshape(H, W, T, C).transpose(2,0,1,3)
    # now galileo_input.shape == (12,64,64,10)
    np.save(f"{output_dir}/{patch_name}.npy", galileo_input)
    return input

for idx, feat in enumerate(samples_fc.getInfo()["features"]):
  print(f"Processing sample {idx}...")
  #for indexes less than 10, make sure it starts with 0 (eg 01, 02 etc)
  if idx < 10:
    patch_name = f"{FILE_PREFIX}_0{idx}"
  else:
    patch_name = f"{FILE_PREFIX}_{idx}"
  process_patch(feat, patch_name)


Processing sample 0...
Processing sample 1...
Processing sample 2...
Processing sample 3...
Processing sample 4...
Processing sample 5...
Processing sample 6...
Processing sample 7...
Processing sample 8...
Processing sample 9...
Processing sample 10...
Processing sample 11...
Processing sample 12...
Processing sample 13...
Processing sample 14...
Processing sample 15...
Processing sample 16...
Processing sample 17...
Processing sample 18...
Processing sample 19...
Processing sample 20...
Processing sample 21...
Processing sample 22...
Processing sample 23...
Processing sample 24...
Processing sample 25...
Processing sample 26...
Processing sample 27...
Processing sample 28...
Processing sample 29...
Processing sample 30...
Processing sample 31...
Processing sample 32...
Processing sample 33...
Processing sample 34...
Processing sample 35...
Processing sample 36...
Processing sample 37...
Processing sample 38...
Processing sample 39...
Processing sample 40...
Processing sample 41...
Pr

In [12]:
!mv galileo galileo_lib

In [13]:
# 1) Imports
import sys
import os
import torch

# Add the galileo_lib directory to the Python path
# This allows relative imports to work correctly outside the package
galileo_lib_path = os.path.abspath(os.path.join(os.getcwd(), "galileo_lib"))  # Use galileo_lib, not galileo_lib/src
if galileo_lib_path not in sys.path:
    sys.path.insert(0, galileo_lib_path)

In [14]:
from pathlib import Path
import torch, glob, numpy as np

# Adjust these imports if your galileo code lives elsewhere
from galileo_lib.src.galileo import Encoder
from galileo_lib.src.data.utils import construct_galileo_input

# 1) Device & model
device    = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_dir = Path("galileo_lib") / "data" / "models" / "nano"
model     = Encoder.load_from_folder(model_dir).eval()

# 2) Find your patches
patch_files = glob.glob("content/patches/veg_patch_*.npy")
print("Found", len(patch_files), "patches")

# 3) Loop: build args dict from mo, plus patch_size
all_embs = []
for fp in patch_files:
    arr = np.load(fp)
    s2 = torch.from_numpy(arr).float().permute(1, 2, 0, 3) # (H, W, T, C)
    # build the MaskedOutput
    mo = construct_galileo_input(s2=s2, normalize=True)

    # cast back to float32 and add batch dim
    s_t_x = mo.space_time_x.float().unsqueeze(0).to(device)
    sp_x  = mo.space_x.float().unsqueeze(0).to(device)
    t_x   = mo.time_x.float().unsqueeze(0).to(device)
    st_x  = mo.static_x.float().unsqueeze(0).to(device)

    inputs = {
        "s_t_x": s_t_x,
        "sp_x":  sp_x,
        "t_x":   t_x,
        "st_x":  st_x,
        # … include your masks and months as before
        "s_t_m": mo.space_time_mask.unsqueeze(0).to(device),
        "sp_m":  mo.space_mask.unsqueeze(0).to(device),
        "t_m":   mo.time_mask.unsqueeze(0).to(device),
        "st_m":  mo.static_mask.unsqueeze(0).to(device),
        "months": mo.months.unsqueeze(0).to(device),
        "patch_size": s2.shape[0]
    }

    # forward‐pass
    with torch.no_grad():
        out = model(**inputs)

    # grab [CLS] token embedding
    out = model(**inputs)

    # out[3] has shape [1, 4, 128]
    # take the CLS embedding at position 0 along the sequence dim:
    last_hidden = out[3]                   # → [1, 4, 128]
    cls_emb     = last_hidden[:, 0, :]     # → [1, 128]
    emb         = cls_emb.cpu().detach().numpy()    # → array shape (1, 128)
    all_embs.append(emb)

# 4) Stack & save
if not all_embs:
    raise RuntimeError("No embeddings extracted; check your paths!")
embs = np.vstack(all_embs)   # (N_patches, D)
print("Embeddings shape:", embs.shape)
np.save("content/patch_embeddings.npy", embs)
print("Saved to content/patch_embeddings.npy")

Found 100 patches
dict_keys([13, 16, 6, 18])
dict_keys([13, 16, 6, 18])
dict_keys([13, 16, 6, 18])
dict_keys([13, 16, 6, 18])
dict_keys([13, 16, 6, 18])
dict_keys([13, 16, 6, 18])
dict_keys([13, 16, 6, 18])
dict_keys([13, 16, 6, 18])
dict_keys([13, 16, 6, 18])
dict_keys([13, 16, 6, 18])
dict_keys([13, 16, 6, 18])
dict_keys([13, 16, 6, 18])
dict_keys([13, 16, 6, 18])
dict_keys([13, 16, 6, 18])
dict_keys([13, 16, 6, 18])
dict_keys([13, 16, 6, 18])
dict_keys([13, 16, 6, 18])
dict_keys([13, 16, 6, 18])
dict_keys([13, 16, 6, 18])
dict_keys([13, 16, 6, 18])
dict_keys([13, 16, 6, 18])
dict_keys([13, 16, 6, 18])
dict_keys([13, 16, 6, 18])
dict_keys([13, 16, 6, 18])
dict_keys([13, 16, 6, 18])
dict_keys([13, 16, 6, 18])
dict_keys([13, 16, 6, 18])
dict_keys([13, 16, 6, 18])
dict_keys([13, 16, 6, 18])
dict_keys([13, 16, 6, 18])
dict_keys([13, 16, 6, 18])
dict_keys([13, 16, 6, 18])
dict_keys([13, 16, 6, 18])
dict_keys([13, 16, 6, 18])
dict_keys([13, 16, 6, 18])
dict_keys([13, 16, 6, 18])
dict_keys(

In [17]:
# 1) Imports
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import glob

# 2) Load embeddings
embeddings = np.load("content/patch_embeddings.npy")   # shape: (N_samples, D)

# 3) Load labels and align by sample_id
df = pd.read_csv("veg_samples_2024.csv")
# assume df has columns: sample_id,label,....
df = df.sort_values("system:index").reset_index(drop=True)
labels = df["label"].values                              # shape: (N_samples,)

assert embeddings.shape[0] == labels.shape[0], \
    f"Num embeddings ({embeddings.shape[0]}) ≠ num labels ({labels.shape[0]})"

# 4) Train/test split
X_train, X_test, y_train, y_test = train_test_split(
    embeddings, labels,
    test_size=0.2,          # 80/20 split
    random_state=42,
    stratify=labels         # preserve class balance
)

# 5) Fit a logistic‐regression classifier
clf = LogisticRegression(max_iter=1000)
clf.fit(X_train, y_train)

# 6) Evaluate on the held‐out set
y_pred = clf.predict(X_test)
print("Accuracy:", accuracy_score(y_test, y_pred))
print("\nClassification report:\n", classification_report(y_test, y_pred))
print("Confusion matrix:\n", confusion_matrix(y_test, y_pred))

# 7) (Optional) 5-fold cross-validation on the full dataset
cv_scores = cross_val_score(clf, embeddings, labels, cv=5, scoring="accuracy")
print(f"\n5-fold CV accuracy: {cv_scores.round(3)} → mean {cv_scores.mean():.3f}")

Accuracy: 0.8

Classification report:
               precision    recall  f1-score   support

           0       0.80      1.00      0.89        16
           1       0.00      0.00      0.00         4

    accuracy                           0.80        20
   macro avg       0.40      0.50      0.44        20
weighted avg       0.64      0.80      0.71        20

Confusion matrix:
 [[16  0]
 [ 4  0]]


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))



5-fold CV accuracy: [0.8  0.8  0.8  0.75 0.75] → mean 0.780
